In [3]:
import pandas as pd
import seaborn as sns
import itertools
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import Counter 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


Code to generate synthetic data of all possible scenarios of Harry kanes next penalty

In [2]:
# Set parameters
isCruciality = [0,1]
IsFriendlyLeagueGame = [0,1]
WasThePlayerSubstitutedOn = [0,1]
PenaltiesTakenInGame = [0,1]

# Previous walkin time for penalty
PrevWalkIn = [507]


CardsInGame = [0, 1, 2]
PenaltyTime = range(1, 91)  # From 1 to 90
GoalDifferenceDuringGoal = [-1, 0, 1]
WalkDuration = range(300, 701, 50)  # From 350 to 800, step 25
FanAdvantage = [0,1,2]
DidPlayerWinPenalty = [0, 1]
isPlayerBooked = [0, 1]
GoalsInGame = [0, 1]
AssistsInGame = [0, 1]

# Create a function to determine if it's the first half based on PenaltyTime
def isFirsthalf(penalty_time):
    return 1 if penalty_time < 46 else 0

# Create a function to determine if it's extra time based on PenaltyTime
def isExtratime(penalty_time):
    return 1 if penalty_time in [45, 90] else 0

# Generate all possible combinations
combinations = list(itertools.product(isCruciality, IsFriendlyLeagueGame, WasThePlayerSubstitutedOn,
                                      PenaltiesTakenInGame, PrevWalkIn, CardsInGame, PenaltyTime,
                                      GoalDifferenceDuringGoal, WalkDuration, FanAdvantage,
                                      DidPlayerWinPenalty, isPlayerBooked, GoalsInGame, AssistsInGame))

# Build the dataframe from the combinations and apply isFirsthalf and isExtratime rules
df_combinations = pd.DataFrame(combinations, columns=[
    'isCruciality', 'IsFriendlyLeagueGame', 'WasThePlayerSubstitutedOn', 'PenaltiesTakenInGame', 'PrevWalkIn',
    'CardsInGame', 'PenaltyTime', 'GoalDifferenceDuringGoal', 'WalkDuration', 'FanAdvantage',
    'DidPlayerWinPenalty', 'isPlayerBooked', 'GoalsInGame', 'AssistsInGame'])


df_combinations['isFirsthalf'] = df_combinations['PenaltyTime'].apply(isFirsthalf)
df_combinations['isExtratime'] = df_combinations['PenaltyTime'].apply(isExtratime)
df_combinations['NoOfMinPlayedByThePlayer'] = df_combinations['PenaltyTime']

# HK previous bucket class
df_combinations['PrevBucket'] = 1

#Current state of HK probabilities
df_combinations['Prob_0'] = 0.166667
df_combinations['Prob_1'] = 0.559524
df_combinations['Prob_2'] = 0.273810

df_combinations.shape

(5598720, 21)

Normalization

In [5]:
# Load the saved scaler
scaler = joblib.load('./models/min_max_scaler.pkl')

In [ ]:
# Columns to normalize
cols_to_normalize = ['NoOfMinPlayedByThePlayer', 'PenaltyTime','WalkDuration','PrevWalkIn','Prob_2','Prob_1','Prob_0']

# Initialize the StandardScaler
#scaler = StandardScaler()

# Apply scaling only to specific columns
df_combinations[cols_to_normalize] = scaler.fit_transform(df_combinations[cols_to_normalize])